# Title

MRI Lesion Segmentation in Multiple Sclerosis Database

# Description

Multiple sclerosis (MS) is the most common inflammatory demyelinating disease of the central nervous system. The disease can appear in multiple types, depending on the duration of the relapse and remitting stages. The initial point of the disease is Clinically Isolated Syndrome (CIS), that is up to 24 hours long. During the following progress, the demyelated regions (so-called lesions) are starting to appear in the brain white matter, causing interconnections between neurons to degrade. 

Multiple tests are used to diagnose MS precisely. Lesion detection and traction through Magnetic Resonanse Imaging (MRI) appears to be pretty sensitive and safe among these methods. The described dataset can be useful to estimate the future progress of the disease since the CIS stage based on MRI scans.

# Experiment

- 38 subjects with diagnosed CIS MS and 20 healthy subjects with different age, gender and degree of disability participated in the experiment.
- MRI was made since 0 month and 6-12 months after the CIS
- EDSS coefficients were estimated for each subject, and were reestimated for MS patients during the second procedure
- Lesions on MR images were segmented manually

### MRI device

Serial longitudinal transversed MR scans was made using a T2-weighted turbo spinecho pulse sequence (repetition time = 4408 ms, echo time = 100 ms, echo spacing = 10.8 ms). The reconstructed image had a slice thickness of 5 mm and a field of view of 230 mm with a pixel resolution of 2.226 pixels/mm. 

MRI is an effective technique to measure MS because MRI uses magnetic fields and radio waves to measure the relative water content in tissues - both normal tissue and abnormal - in the body. Since the layer of myelin the protects the nerve cell is fatty, it repels water. In the areas where myelin has been damaged by MS, the fat is stripped away. With the fat gone, the area holds more water, which shows in MRI either as a bright spot or darkened areas depending on the type of scan that’s used.

### Subjects

Thirty-eight patients (17 males, and 21 females), aged 34.1 ± 10.5 (mean age ± standard deviation), with a CIS of MS and MRI-detectable brain lesions were scanned twice at 1.5 T with an interval of 6-12 months. Additionally, the brains of 20 healthy, age-matched (mean ± SD: 30.8 ± 7.6) volunteers (8 males, and 12 females) were MRI-scanned to allow segmentation and analysis of normal brain white matter 

### Target
The target variable is EDSS score (Expanded Disability Status Scale) in a range from 0 to 10, providing a way of quantifying disability in MS and monitoring the changes over time in level of disability. 
- 0.5 unit increments
- EDSS steps 1.0 to 4.5 refer to people with MS who are able to walk without any aid
- EDSS steps 5.0 to 9.5 are defined by the impairment to walking

Here we are specifically interested in EDSS score > 2 as a cut-off point because above this point neurological signs demonstrate the onset of accumulating disability.
- EDSS(2) -> 20% Stabilized
- EDSS(5) -> 50% Stabilized

### File content
- 1887 MR images in TIFF or BMP format
- 1777 segmented lesions in .plq format. Each file contains lesion boundaries, cropped lesion image, related binary mask
- The data about healthy subjects and gender data are missing

# Hypotheses

1. Shape Features

    1. Maximum length of x and y coords, 
    1. Area, 
    1. perimeter, 
    1. major and minor axis length, 
    1. eccentricity, 
    1. convex area 
    1. volume
    
These features tend to describe the shape and size of lesion. Overall an increment in these directly leads to number of demyelinated axons leads to an increased signal transmission impairment, i.e. patient EDSS score can increase.

2. Texture features

    1. Median Value - intuitively shows the brightness of each ROI. MS Lesions are brighter than NWM.
    1. Contrast - which is a measure of local variation between pixel intensities. MS regions have higher contrast values than NWM regions
    1. Inverse different moment (IDM) – This is related to lesions homogeneity. In NWM, IDM had higher values than in MS regions, implying that NWM were smoother and more homogeneous.
    1. Entropy and Sum of average  - MS lesions attained lower values than NWM ROIs, indicating that the degree of randomness of pixel intensities or textural roughness in MS regions was lower.
    
(TODO provide causal relations with MS disease)
    
3. Spatial features

    1. Lesion location grid

The location of the lesion can be connected with the EDSS score, because it is important what parts of brain were damaged by the inflammation. 

# Data and theory comparison

This section describes differences between theoretical knowledges about the disease, placed as the set of statistical hypotheses, and the results of statistical tests against an actual data. The inconsistencies are described in the sections above

## Location

The goal of the investigation was to find regions in a human brain which, affected by lesions, leads to a serious degree of disability. I.e. is there a significant difference in EDSS score for MS patients with lesions in white matter near the cortex regions responsible for speech, high-level thinking, tactile sensations and movements, and MS patients without ones.

To check it, we've splitted patients on two groups with EDSS > 2 and EDSS $\leq$ 2. Then we've splitted each MRI scan into grid with the fixed cell size. For each cell position among all MRI scans from the same Z slice, we've performed two-sampled homogenity test (Mann-Whitney U). As a result we've got cells close to precentral gyrus, their p-value was slightly less than 5%. When we've performed such test for the lesser size of grid cell, there were no regions with comparable significance.

![](./images/first-iteration.png)
<center>Fig. 1. Cells with high statistical significance mostly covers the region near right precentral gyrus</center>

To improve the results, we've investigated the dataset carefully, and found out the following defects:
- There were MRI scans with different resolutions in the dataset. Before the next iteration, all of the scans were resized to 512x512
- The order of Z slices was broken for a couple of patients. As an example, for patient SE one of the last scans in his first session should be treated as the first one. To fix this, a simple file moving script was written.
- The number of Z slices were different for each session
- The geometric proportions of each skull were different for each patient
- Sometimes a small rotation of the skull was presented on MRI scans

To deal with the last three points, we've implemented the following algorithm:
- First of all, the average proportions of the skull were restored. To do this, we've gathered all scans on each Z level, and then took an average value for each pixel. To deal with different number of slices for each patient, a new Z level for each scan were specified with a following formula:
$$ Z^{new}_i = 4[\frac{25Z_i}{\max\limits Z_i}]$$
Where Z is an index of a scan, $[x]$ represents ceil operation.
- Then the boundaries of the skull were estimated. The overall process is shown on the Fig. 2. To do that, we've created a filling mask from the (0, 0) point, then removed small objects (for example, text with scan name), highlighted the spot edges and remove all nonzero points except start and end point for each image column to get rid of the inner edges. As a result, we've got an array of (x, y) points defined a skull boundary.
- The boundaries were estimated for the scan we need to transform and for the related model slice. Then we've found the proper mapping $m$ between the boundary points between both groups:
$$ \arg \min \limits_m \sum_i (p^{model}_{m[i]} - p^{scan}_i)^2 $$
This task was solved using hungarian algorithm. As a result, we've got a set of scan points $X$ and the related model points $Y$. 
- We've found a proper affine transformation matrix as the weights and the bias of linear regression.
- The coordinates of each lesion on a scan were transformed using the matrix received above - the result of such affine transformation in shown on the Fig. 3. Then the statistical test described previously was repeated.

![](./images/boundaries.png)
<center>Fig. 2. The process of boundaries extraction. From left to right: original scan, filling procedure, removing small objects, highlighting boundaries, simple heuristics to remove inner structure</center>

![](./images/transform.png)
<center>Fig. 3. The process of scan transformation. First image - original MRI scan, which is slightly rotated and not scaled. Second image - scan after the transformation. Third image - the averaged model of the skull for the related Z slice</center>

As a result, we've received dense clusters consisted of small grid cells with high significance (Mann-Whitney U p-value less than 2%) in a white matter near the following cortex regions:

### Medial and middle frontal gyrus
Medial and middle frontal gyrus are both parts of frontal lobe. There is some evidence that medial frontal gyrus plays a role in executive mechanisms, the same time as middle frontal gyrus  has been connected with processes as different as contemplating distance, recognition of known faces, and accessing word meaning while reading. 
![](./images/medial-frontal.png)
<center>Fig. 4. Statistically significant cells located densely around the region near medial frontal gyrus</center>

### Precentral and postcentral gyrus
Precentral gyrus is the part of the primary motor cortex. Lesions of the precentral gyrus result in paralysis of the contralateral side of the body. 
![](./images/right-precentral-postcentral.png)
<center>Fig. 5. Statistically significant cells are located densely near right precentral and postcentral gyrus</center>

Postcentral gyrus is the part of primary somatosensory cortex. Lesions of the postcentral gyrus result in anesthesia or hyperesthesia of the related contralateral body parts.
![](./images/precentral-postcentral.png)
<center>Fig. 6. Statistically significant cells are located in a white matter near left and right postcentral gyrus. Small region near medial frontal gyrus is highlighted as well</center>

# Related articles

1. C.P. Loizou, V. Murray, M.S. Pattichis, I. Seimenis, M. Pantziaris, C.S. Pattichis, Multi-scale amplitude modulation-frequency modulation (AM-FM) texture analysis of multiple sclerosis in brain MRI images, IEEE Trans. Inform. Tech. Biomed., vol. 15, no. 1, pp. 119-129, 2011.   

1. C.P. Loizou, E.C. Kyriacou, I. Seimenis, M. Pantziaris, S. Petroudi, M. Karaolis, C.S. Pattichis, Brain white matter lesion classification in multiple sclerosis subjects for the prognosis of future disability, Intelligent Decision Technologies Journal (IDT), vol. 7, pp. 3-10, 2013.

1. C.P. Loizou, M. Pantziaris, C.S. Pattichis, I. Seimenis, Brain MRI Image normalization in texture analysis of multiple sclerosis, J. Biomed. Graph. & Comput., vol. 3, no.1, pp. 20-34, 2013. 

1. C.P. Loizou, S. Petroudi, I. Seimenis, M. Pantziaris, C.S. Pattichis, Quantitative texture analysis of brain 
white matter lesions derived from T2-weighted MR images in MS patients with clinically isolated syndrome, J. Neuroradiol., acepted. 

1. https://www.nationalmssociety.org/Symptoms-Diagnosis/Diagnosing-Tools/MRI

1. Purves, Dale. Neuroscience. Sunderland, Mass: Sinauer Associates, Publishers, 2004.